##Tune HP

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
wandb.login()
%env WANDB_PROJECT=assitant_summarization_sweeps
%env WANDB_LOG_MODEL=end

In [ ]:
!git clone https://github.com/spbu-smart-assistant/support-chatbot.git

In [ ]:
!pip install --no-cache-dir -q transformers sentencepiece

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
our_dataset = load_dataset('json', data_files='support-chatbot/data/raw/summary_data.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summa"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def summarize(
    text, n_words=None, compression=None,
    max_length=1500, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
tokenized_our_dataset = our_dataset.map(preprocess_function, batched=True)
len_dataset = len(tokenized_our_dataset['train'])

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_NAME)

In [ ]:
from transformers import T5Config, AutoModelForSeq2SeqLM
dropout = 0.1
def model_init():
  t5_config = T5Config.from_pretrained(MODEL_NAME)
  t5_config.dropout_rate = dropout
  t5_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME,
                                                        config=t5_config)
  return t5_model

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'avg_rouge_f',
        'goal': 'maximize'
    }
}

# hyperparameters
parameters_dict = {
    'epochs': {
        'value': 100
        },
    'batch_size': {
        'values': [1, 2, 3, 4]
        },
    'learning_rate': {
        'distribution': 'uniform',
        'min': 1e-7,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
    },
    'dropout': {
        'values': [0.0, 0.1, 0.2, 0.3]
    }
}
sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='assistant_summarization_sweeps')

In [ ]:
!pip install -q rouge

In [ ]:
from rouge import Rouge
import numpy as np
rouge = Rouge()
def compute_metrics_fn(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print('True:', decoded_labels)
    print('prediction:', decoded_preds)

    scores = rouge.get_scores(decoded_labels, decoded_preds, avg=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    scores['gen_len'] = {'f': np.mean(prediction_lens)}
    scores['avg_rouge_f'] = {'f': np.mean([scores['rouge-1']['f'], 
                                           scores['rouge-2']['f'], 
                                           scores['rouge-l']['f']])}
    scores['sum_rouge_f'] = {'f': np.sum([scores['rouge-1']['f'], 
                                           scores['rouge-2']['f'], 
                                           scores['rouge-l']['f']])}  

    return {k: round(v['f'], 4) for k, v in scores.items()}

In [ ]:
from transformers import EarlyStoppingCallback
callback = EarlyStoppingCallback(early_stopping_patience=5, 
                                 early_stopping_threshold=0.0001)

In [ ]:
import torch
from tqdm.auto import trange
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
def train(config=None):
  with wandb.init(config=config):
    avg_rouge_f = 0
    for i in trange(len_dataset):
      ex_text = [tokenized_our_dataset['train'][i]['text']]
      ex_summa = [tokenized_our_dataset['train'][i]['summa']]
      ex_input_ids = [tokenized_our_dataset['train'][i]['input_ids']]
      ex_attention_mask = [tokenized_our_dataset['train'][i]['attention_mask']]
      ex_labels = [tokenized_our_dataset['train'][i]['labels']]

      test_dataset = Dataset.from_dict({'text': ex_text, 'summa': ex_summa, 
                                        'input_ids': ex_input_ids, 
                                        'attention_mask': ex_attention_mask,
                                        'labels': ex_labels})
      
      train_text = tokenized_our_dataset['train'][:i]['text'] + tokenized_our_dataset['train'][i+1:]['text']
      train_summa = tokenized_our_dataset['train'][:i]['summa'] + tokenized_our_dataset['train'][i+1:]['summa']
      train_input_ids = tokenized_our_dataset['train'][:i]['input_ids'] + tokenized_our_dataset['train'][i+1:]['input_ids']
      train_attention_mask = tokenized_our_dataset['train'][:i]['attention_mask'] + tokenized_our_dataset['train'][i+1:]['attention_mask']
      train_labels = tokenized_our_dataset['train'][:i]['labels'] + tokenized_our_dataset['train'][i+1:]['labels']

      train_dataset = Dataset.from_dict({'text': train_text, 'summa': train_summa, 
                                        'input_ids': train_input_ids, 
                                        'attention_mask': train_attention_mask,
                                        'labels': train_labels})
      config = wandb.config
      training_args = Seq2SeqTrainingArguments(
          output_dir="my_awesome_model",
          report_to='wandb',
          evaluation_strategy="steps",
          eval_steps=20,
          learning_rate=config.learning_rate,
          per_device_train_batch_size=config.batch_size, 
          per_device_eval_batch_size=1,
          weight_decay=config.weight_decay,
          warmup_steps = 50,
          save_total_limit=3,
          num_train_epochs=config.epochs,
          predict_with_generate=True,
          push_to_hub=False,
          logging_steps=20,
          load_best_model_at_end=True,
      )
      global dropout
      dropout = config.dropout
      trainer = Seq2SeqTrainer(
          model_init=model_init,
          args=training_args,
          data_collator=data_collator,
          tokenizer=tokenizer,
          train_dataset=train_dataset,
          eval_dataset=test_dataset,
          compute_metrics=compute_metrics_fn
      )
      trainer.add_callback(callback)
      trainer.train()
      metrics = trainer.evaluate()
      print('metrics:', metrics)
      avg_rouge_f += metrics['eval_sum_rouge_f']
      torch.cuda.empty_cache()
    wandb.log({'avg_rouge_f': avg_rouge_f / len_dataset})
    %rm -rf wandb my_awesome_model

In [ ]:
wandb.agent(sweep_id, train, count=15)

wandb: Agent Starting Run: xb5fu6o5 with config:
wandb: 	batch_size: 3
wandb: 	dropout: 0.3
wandb: 	epochs: 20
wandb: 	learning_rate: 0.0002413268418140153
wandb: 	weight_decay: 0.5
wandb: Currently logged in as: nalex22-uv-j02. Use `wandb login --relogin` to force relogin


  0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
20,6.512300,1.080096,0.000000,0.000000,0.000000,11.000000,0.000000
40,4.795000,0.979685,0.352900,0.266700,0.352900,16.000000,0.324200
60,4.223900,1.019287,0.400000,0.307700,0.400000,13.000000,0.369200
80,3.583000,0.980278,0.400000,0.307700,0.400000,13.000000,0.369200
100,3.312700,0.970968,0.400000,0.307700,0.400000,13.000000,0.369200


True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['Спасибо, я и я.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['У пользователя в компьютере не получается пароль от почты.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['У пользователя не получается пароль от почты.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['У пользователя проблемы с пароль от почты.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.']
prediction: ['У пользователя проблемы с пароль от почты.']


...

wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss


Run 4jtsjzhp errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 34.00 MiB (GPU 0; 14.75 GiB total capacity; 12.78 GiB already allocated; 6.81 MiB free; 13.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run 4jtsjzhp errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 34.00 MiB (GPU 0; 14.75 GiB total capacity; 12.78 GiB already allocated; 6.81 MiB free; 13.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: ux2ati97 with config:
wandb: 	batch_size: 4
wandb: 	dropout: 0
wandb: 	epochs: 20
wandb: 	learning_rate: 0.0007613841978311972
wandb: 	weight_decay: 0


  0%|          | 0/16 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


##Train ans test

In [ ]:
!pip install --no-cache-dir -q transformers sentencepiece

In [ ]:
!git clone https://github.com/spbu-smart-assistant/support-chatbot.git

In [23]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config
MODEL_NAME = 'bragovo/qqq'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [8]:
!pip install -q datasets

In [6]:
from datasets import load_dataset

In [ ]:
our_dataset = load_dataset('json', data_files='support-chatbot/data/raw/summary_data.json')

In [10]:
our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa'],
        num_rows: 25
    })
})

In [11]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summa"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_our_dataset = our_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_NAME)

In [15]:
!pip install -q rouge

In [17]:
from rouge import Rouge
import numpy as np
rouge = Rouge()
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print('True:', decoded_labels)
    print('prediction:', decoded_preds)

    scores = rouge.get_scores(decoded_labels, decoded_preds, avg=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    scores['gen_len'] = {'f': np.mean(prediction_lens)}
    scores['avg_rouge_f'] = {'f': np.mean([scores['rouge-1']['f'], 
                                           scores['rouge-2']['f'], 
                                           scores['rouge-l']['f']])}  

    return {k: round(v['f'], 4) for k, v in scores.items()}

In [18]:
from transformers import EarlyStoppingCallback
callback = EarlyStoppingCallback(early_stopping_patience=5, 
                                 early_stopping_threshold=0.0001)

In [20]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
batch_size = 3
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_model",
    evaluation_strategy="steps",
    eval_steps=25,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    warmup_steps = 50,
    save_total_limit=3,
    num_train_epochs=100,
    predict_with_generate=True,
    push_to_hub=False,
    logging_steps=25,
    load_best_model_at_end=True,
)

In [21]:
len_dataset = len(tokenized_our_dataset['train'])

In [24]:
from datasets import Dataset
from tqdm.auto import trange
t5_config = T5Config.from_pretrained(MODEL_NAME)
t5_config.dropout_rate = 0.1

for i in trange(len_dataset):
  model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME,
                                                     config=t5_config)

  ex_text = [tokenized_our_dataset['train'][i]['text']]
  ex_summa = [tokenized_our_dataset['train'][i]['summa']]
  ex_input_ids = [tokenized_our_dataset['train'][i]['input_ids']]
  ex_attention_mask = [tokenized_our_dataset['train'][i]['attention_mask']]
  ex_labels = [tokenized_our_dataset['train'][i]['labels']]

  test_dataset = Dataset.from_dict({'text': ex_text, 'summa': ex_summa, 
                                    'input_ids': ex_input_ids, 
                                    'attention_mask': ex_attention_mask,
                                    'labels': ex_labels})
  
  train_text = tokenized_our_dataset['train'][:i]['text'] + tokenized_our_dataset['train'][i+1:]['text']
  train_summa = tokenized_our_dataset['train'][:i]['summa'] + tokenized_our_dataset['train'][i+1:]['summa']
  train_input_ids = tokenized_our_dataset['train'][:i]['input_ids'] + tokenized_our_dataset['train'][i+1:]['input_ids']
  train_attention_mask = tokenized_our_dataset['train'][:i]['attention_mask'] + tokenized_our_dataset['train'][i+1:]['attention_mask']
  train_labels = tokenized_our_dataset['train'][:i]['labels'] + tokenized_our_dataset['train'][i+1:]['labels']

  train_dataset = Dataset.from_dict({'text': train_text, 'summa': train_summa, 
                                    'input_ids': train_input_ids, 
                                    'attention_mask': train_attention_mask,
                                    'labels': train_labels})
  trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
  trainer.add_callback(callback)
  trainer.train()
  torch.cuda.empty_cache()

  0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
25,2.166800,0.896459,0.000000,0.000000,0.000000,19.000000,0.000000
50,1.573400,0.705029,0.000000,0.000000,0.000000,17.000000,0.000000
75,1.141300,0.622708,0.000000,0.000000,0.000000,13.000000,0.000000
100,0.725500,0.597052,0.000000,0.000000,0.000000,13.000000,0.000000
125,0.630100,0.570725,0.500000,0.428600,0.500000,19.000000,0.476200
150,0.517700,0.532356,0.500000,0.428600,0.500000,19.000000,0.476200
175,0.415300,0.519096,0.500000,0.428600,0.500000,19.000000,0.476200
200,0.331600,0.515260,0.500000,0.428600,0.500000,19.000000,0.476200
225,0.248400,0.514578,0.500000,0.428600,0.500000,19.000000,0.476200
250,0.259900,0.470839,0.500000,0.428600,0.500000,19.000000,0.476200


...

Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
25,2.139900,1.458471,0.375000,0.142900,0.250000,18.000000,0.256000
50,1.496400,1.232911,0.375000,0.142900,0.250000,18.000000,0.256000
75,1.049000,1.142966,0.375000,0.142900,0.250000,18.000000,0.256000
100,0.796100,1.136701,0.375000,0.142900,0.250000,18.000000,0.256000
125,0.604800,1.153849,0.000000,0.000000,0.000000,19.000000,0.000000
150,0.474700,1.123150,0.000000,0.000000,0.000000,19.000000,0.000000
175,0.407500,1.208962,0.000000,0.000000,0.000000,19.000000,0.000000
200,0.307500,1.432068,0.000000,0.000000,0.000000,19.000000,0.000000
225,0.266400,1.479953,0.000000,0.000000,0.000000,19.000000,0.000000
250,0.223100,1.614926,0.000000,0.000000,0.000000,19.000000,0.000000


True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Компьютерный класс на факультете ПМ-ПУ, аудитория 203.']
True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Компьютерный класс на факультете ПМ-ПУ, аудитория 203.']
True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Компьютерный класс на факультете ПМ-ПУ, аудитория 203.']
True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Компьютерный класс на факультете ПМ-ПУ, аудитория 203.']
True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Пользователь хочет сообщить о проблеме с оборудованием компьютерного класса.']
True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Пользователь хочет сообщить о проблеме с оборудованием компьютерного класса.']
True: ['В аудитории 203 на факультете ПМ-ПУ компьютер не работает.']
prediction: ['Пользователь хочет сообщить о про

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
25,2.120700,1.546163,0.000000,0.000000,0.000000,7.000000,0.000000
50,1.524700,1.247074,0.166700,0.000000,0.166700,13.000000,0.111100
75,1.045900,1.099882,0.461500,0.000000,0.307700,16.000000,0.256400
100,0.777700,1.040208,0.461500,0.000000,0.307700,16.000000,0.256400
125,0.616800,1.012317,0.461500,0.000000,0.307700,16.000000,0.256400
150,0.487300,0.973909,0.600000,0.250000,0.600000,9.000000,0.483300
175,0.407700,0.884285,0.600000,0.250000,0.600000,9.000000,0.483300
200,0.331800,0.773668,0.600000,0.250000,0.600000,9.000000,0.483300
225,0.276700,0.651725,0.500000,0.200000,0.500000,13.000000,0.400000
250,0.240000,0.629190,0.600000,0.250000,0.600000,9.000000,0.483300


True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Спасибо за помощь.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Это служба технической поддержки университета.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Компания технической поддержки университета хочет отчислиться.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Компания технической поддержки университета хочет отчислиться.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Компания технической поддержки университета хочет отчислиться.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Пользователь хочет отчислиться.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Пользователь хочет отчислиться.']
True: ['Пользователь хочет знать, как отчислиться из университета.']
prediction: ['Пол